# Image-Based Baseline Model

Using transfer learning with ResNet50 to predict biomass directly from images.

**Why images only?** The test set only provides images - no NDVI, Height, State, or Species data!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from tqdm import tqdm

sns.set_style('whitegrid')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

## 1. Load and Prepare Data

In [ ]:
# Load training data
train_df = pd.read_csv('competition/train.csv')

# Convert to wide format (one row per image)
train_wide = train_df.pivot_table(
    index=['image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'],
    columns='target_name',
    values='target'
).reset_index()

print(f"Total images: {len(train_wide)}")
print(f"Target columns: {['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']}")
train_wide.head()

In [ ]:
# Target columns
target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

# Split into train and validation
train_data, val_data = train_test_split(train_wide, test_size=0.2, random_state=42)

print(f"Training images: {len(train_data)}")
print(f"Validation images: {len(val_data)}")

## 2. Create PyTorch Dataset

In [ ]:
class PastureDataset(Dataset):
    def __init__(self, dataframe, root_dir='competition', transform=None, target_cols=None):
        """
        Args:
            dataframe: DataFrame with image_path and target columns
            root_dir: Directory containing the images
            transform: Optional transforms to apply to images
            target_cols: List of target column names
        """
        self.dataframe = dataframe.reset_index(drop=True)
        self.root_dir = root_dir
        self.transform = transform
        self.target_cols = target_cols
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        # Get image path
        img_path = os.path.join(self.root_dir, self.dataframe.loc[idx, 'image_path'])
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        # Get targets
        targets = self.dataframe.loc[idx, self.target_cols].values.astype('float32')
        targets = torch.tensor(targets, dtype=torch.float32)
        
        return image, targets

In [ ]:
# Define image transforms
# Using ImageNet normalization since we're using pre-trained ResNet
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224
    transforms.RandomHorizontalFlip(),  # Simple augmentation
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = PastureDataset(train_data, transform=train_transform, target_cols=target_cols)
val_dataset = PastureDataset(val_data, transform=val_transform, target_cols=target_cols)

# Create dataloaders
batch_size = 16
# Note: num_workers=0 to avoid multiprocessing issues in Jupyter notebooks
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

In [ ]:
# Test the dataloader - visualize a batch
sample_images, sample_targets = next(iter(train_loader))
print(f"Image batch shape: {sample_images.shape}")  # [batch_size, 3, 224, 224]
print(f"Target batch shape: {sample_targets.shape}")  # [batch_size, 5]
print(f"\nSample targets (first image):")
for i, col in enumerate(target_cols):
    print(f"  {col}: {sample_targets[0, i]:.2f}g")

## 3. Build Model with Pre-trained ResNet50

In [ ]:
class BiomassPredictor(nn.Module):
    def __init__(self, num_outputs=5, pretrained=True):
        super(BiomassPredictor, self).__init__()
        
        # Load pre-trained ResNet50
        self.resnet = models.resnet50(pretrained=pretrained)
        
        # Get the number of features from ResNet's final layer
        num_features = self.resnet.fc.in_features
        
        # Replace the final fully connected layer
        # ResNet outputs 1000 classes by default, we need 5 continuous outputs
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_outputs)  # 5 outputs for our 5 targets
        )
        
    def forward(self, x):
        return self.resnet(x)

# Create model
model = BiomassPredictor(num_outputs=5, pretrained=True)
model = model.to(device)

print("Model created!")
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 4. Training Setup

In [ ]:
# Loss function - Mean Squared Error for regression
criterion = nn.MSELoss()

# Optimizer - Adam with learning rate
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Learning rate scheduler - reduce LR when validation loss plateaus
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

print(f"Learning rate: {learning_rate}")
print(f"Optimizer: Adam")
print(f"Loss function: MSE")

## 5. Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for images, targets in tqdm(loader, desc='Training'):
        images = images.to(device)
        targets = targets.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * images.size(0)
    
    return total_loss / len(loader.dataset)

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for images, targets in tqdm(loader, desc='Validation'):
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item() * images.size(0)
            
            all_predictions.append(outputs.cpu().numpy())
            all_targets.append(targets.cpu().numpy())
    
    all_predictions = np.vstack(all_predictions)
    all_targets = np.vstack(all_targets)
    
    return total_loss / len(loader.dataset), all_predictions, all_targets

In [ ]:
# Training loop
num_epochs = 3  # Start with 3 epochs for quick baseline
best_val_loss = float('inf')
train_losses = []
val_losses = []

print(f"Training for {num_epochs} epochs...\n")

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # Validate
    val_loss, val_predictions, val_targets = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    # Update learning rate
    scheduler.step(val_loss)
    
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"✓ Saved best model (val_loss: {val_loss:.4f})")
    
    print("-" * 60)

print("\nTraining complete!")

In [ ]:
# Plot training curves
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss', marker='o')
plt.plot(val_losses, label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Evaluate Model Performance

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_model.pth'))
print("Loaded best model")

# Get predictions on validation set
val_loss, val_predictions, val_targets = validate_epoch(model, val_loader, criterion, device)

print(f"\nBest validation loss: {val_loss:.4f}")

In [ ]:
# Calculate R² scores and competition metric
def calculate_competition_score(y_true, y_pred, target_cols):
    """
    Calculate weighted R² score as per competition rules
    """
    weights = {
        'Dry_Green_g': 0.1,
        'Dry_Dead_g': 0.1,
        'Dry_Clover_g': 0.1,
        'GDM_g': 0.2,
        'Dry_Total_g': 0.5
    }
    
    print("\n" + "="*60)
    print("VALIDATION PERFORMANCE")
    print("="*60)
    
    r2_scores = {}
    total_score = 0
    
    for i, col in enumerate(target_cols):
        r2 = r2_score(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        r2_scores[col] = r2
        
        print(f"\n{col}:")
        print(f"  R² Score: {r2:.4f} (weight: {weights[col]})")
        print(f"  MAE: {mae:.2f}g")
        print(f"  Weighted contribution: {weights[col] * r2:.4f}")
        
        total_score += weights[col] * r2
    
    print("\n" + "="*60)
    print(f"COMPETITION SCORE (Weighted R²): {total_score:.4f}")
    print("="*60)
    
    return r2_scores, total_score

r2_scores, competition_score = calculate_competition_score(val_targets, val_predictions, target_cols)

In [ ]:
# Visualize predictions vs actuals
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, col in enumerate(target_cols):
    ax = axes[idx]
    
    # Scatter plot
    ax.scatter(val_targets[:, idx], val_predictions[:, idx], alpha=0.5)
    
    # Perfect prediction line
    min_val = min(val_targets[:, idx].min(), val_predictions[:, idx].min())
    max_val = max(val_targets[:, idx].max(), val_predictions[:, idx].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect')
    
    ax.set_xlabel(f'Actual {col}')
    ax.set_ylabel(f'Predicted {col}')
    ax.set_title(f'{col}\nR² = {r2_scores[col]:.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

## 7. Prediction Consistency Check

In [ ]:
# Check if predictions respect mathematical relationships
pred_df = pd.DataFrame(val_predictions, columns=target_cols)

# Total should equal Green + Dead + Clover
pred_df['calc_total'] = pred_df['Dry_Green_g'] + pred_df['Dry_Dead_g'] + pred_df['Dry_Clover_g']
pred_df['total_diff'] = pred_df['Dry_Total_g'] - pred_df['calc_total']

# GDM should equal Green + Clover
pred_df['calc_gdm'] = pred_df['Dry_Green_g'] + pred_df['Dry_Clover_g']
pred_df['gdm_diff'] = pred_df['GDM_g'] - pred_df['calc_gdm']

print("Prediction Consistency:")
print(f"\nDry_Total_g vs sum of components:")
print(f"  Mean difference: {pred_df['total_diff'].mean():.2f}g")
print(f"  Std difference: {pred_df['total_diff'].std():.2f}g")
print(f"  Max absolute difference: {pred_df['total_diff'].abs().max():.2f}g")

print(f"\nGDM_g vs (Green + Clover):")
print(f"  Mean difference: {pred_df['gdm_diff'].mean():.2f}g")
print(f"  Std difference: {pred_df['gdm_diff'].std():.2f}g")
print(f"  Max absolute difference: {pred_df['gdm_diff'].abs().max():.2f}g")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(pred_df['total_diff'], bins=30, edgecolor='black')
axes[0].set_xlabel('Dry_Total_g - (Green + Dead + Clover)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Total Biomass Consistency')
axes[0].axvline(0, color='red', linestyle='--', label='Perfect')
axes[0].legend()

axes[1].hist(pred_df['gdm_diff'], bins=30, edgecolor='black')
axes[1].set_xlabel('GDM_g - (Green + Clover)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('GDM Consistency')
axes[1].axvline(0, color='red', linestyle='--', label='Perfect')
axes[1].legend()

plt.tight_layout()
plt.show()

## 8. Generate Test Predictions

Create submission file for the test set.

In [ ]:
# Load test data
test_df = pd.read_csv('competition/test.csv')
print(f"Test samples: {len(test_df)}")
print(f"Unique test images: {test_df['image_path'].nunique()}")
test_df.head()

In [ ]:
# Create test dataset (without targets)
class TestPastureDataset(Dataset):
    def __init__(self, image_paths, root_dir='competition', transform=None):
        self.image_paths = image_paths
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

# Get unique test images
test_images = test_df['image_path'].unique()
print(f"Predicting on {len(test_images)} test images")

# Create dataset and loader
test_dataset = TestPastureDataset(test_images, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# Generate predictions
model.eval()
test_predictions = []

with torch.no_grad():
    for images in tqdm(test_loader, desc='Predicting on test set'):
        images = images.to(device)
        outputs = model(images)
        test_predictions.append(outputs.cpu().numpy())

test_predictions = np.vstack(test_predictions)
print(f"\nTest predictions shape: {test_predictions.shape}")
print(f"Expected: ({len(test_images)}, 5)")

In [ ]:
# Create submission file
# Map predictions back to the required format
pred_dict = {}
for idx, img_path in enumerate(test_images):
    for col_idx, col in enumerate(target_cols):
        pred_dict[img_path + '__' + col] = test_predictions[idx, col_idx]

# Create submission
submission = test_df.copy()
submission['target'] = submission.apply(
    lambda row: pred_dict.get(row['image_path'] + '__' + row['target_name'], 0.0),
    axis=1
)

# Ensure no negative predictions
submission['target'] = submission['target'].clip(lower=0)

# Save
submission[['sample_id', 'target']].to_csv('submission_image_baseline.csv', index=False)
print("\nSubmission saved to submission_image_baseline.csv")
print(f"Submission shape: {submission.shape}")
print("\nFirst 10 rows:")
print(submission[['sample_id', 'target']].head(10))

## Summary

### Image-Based Baseline Results:

**Model:** ResNet50 (pre-trained on ImageNet) with custom regression head

**Key Points:**
- Uses ONLY images (no tabular features) since test set only provides images
- Transfer learning from ImageNet helps with limited training data (357 images)
- Simple data augmentation (flips, rotation, color jitter)
- Predicts all 5 targets simultaneously

**Check the results above:**
- What's your validation competition score?
- Which targets are harder to predict from images alone?
- Are predictions mathematically consistent?

### Potential Improvements:
1. **Better architecture** - Try EfficientNet, Vision Transformer
2. **More augmentation** - More aggressive transforms
3. **Enforce consistency** - Post-process to ensure Total = sum of components
4. **Ensemble** - Train multiple models and average
5. **Larger input size** - 224x224 might lose detail, try 384x384
6. **Fine-tune earlier layers** - Unfreeze more of ResNet
7. **Better loss function** - Weight Dry_Total_g more heavily in loss
8. **Cross-validation** - More robust evaluation